My original data source was removed from Kaggle. I download numerous files from the NC Department of Public Instruction and identified 30 files that would be beneficial. I tried to develop a function to bring all of the files in and edit them. However, each file needed specific changes, so this was not an area to implement automation.

In [1]:
#Importing necessary packages
import pandas as pd
import string
from pandasql import sqldf

In [33]:
#Editing grad4 and grad5 data for merging

#grad4
grad4=pd.read_excel('4yGraduation.xlsx')
#pulling year 2017
grad4=grad4[grad4['reporting_year']==2017]
#pulling grad rate for all youth
grad4=grad4[grad4['subgroup']=='ALL']
#renaming columns for full merge
grad4.rename(columns={'school_code': 'agency_code', 'pct': 'GradRate4Yr'}, inplace=True)
#dropping unimportant columns
grad4.drop(columns=['reporting_year', 'cgr_type', 'subgroup', 'denominator', 'school_name', 'grade_span', 'sbe_region'], 
           inplace=True)
#replacing values
grad4['GradRate4Yr'].replace(['>95', '<5', '*'], [95, 5, 0], inplace=True)
#Getting rid of LEA info
grad4=grad4[~grad4['agency_code'].str.endswith('LEA')]
#setting index
grad4.set_index('agency_code', inplace=True)
#dropping State of North Carolina
grad4.drop(grad4.tail(1).index, inplace=True)



#grad5
grad5=pd.read_excel('5yGraduation.xlsx')
#pulling year 2017
grad5=grad5[grad5['reporting_year']==2017]
#pulling grad rate for all youth
grad5=grad5[grad5['subgroup']=='ALL']
#renaming columns for full merge
grad5.rename(columns={'school_code': 'agency_code', 'pct': 'GradRate5Yr'}, inplace=True)
#dropping unimportant columns
grad5.drop(columns=['reporting_year', 'cgr_type', 'subgroup', 'denominator', 'school_name', 'grade_span', 'sbe_region'], 
           inplace=True)
#replacing values
grad5['GradRate5Yr'].replace(['>95', '<5', '*'], [95, 5, 0], inplace=True)
#Getting rid of LEA info
grad5=grad5[~grad5['agency_code'].str.endswith('LEA')]
#setting index
grad5.set_index('agency_code', inplace=True)
#dropping State of North Carolina
grad5.drop(grad5.tail(1).index, inplace=True)

In [34]:
#Free and Reduced Lunch
FreeRed=pd.read_excel('FreeReducedLunch.xlsx')
#Concatenate  ID and School ID to get agency_code
FreeRed['SFA #']=FreeRed['SFA #'].apply(str)
FreeRed['SFA #'].replace(['10', '20', '30', '40', '50', '60', '70', '80', '90'], 
                         ['010', '020', '030', '040', '050', '060', '070', '080', '090'],
                         inplace=True)
FreeRed['Site #']=FreeRed['Site #'].apply(str)
FreeRed['agency_code']=FreeRed['SFA #']+FreeRed['Site #']
#Creating CEP column
FreeRed.rename(columns={'Provision': 'CEP_Eligible', '%EDS': 'EDS%', 'SFA Name': 'LEA_Name'}, inplace=True)
FreeRed['CEP_Eligible'].replace({'CEP': 1}, inplace=True)
#Filling in 0s for missing Eligible Values
FreeRed['CEP_Eligible'].fillna('0', inplace=True)
#Dropping unnecessary columns
FreeRed.drop(columns=['SFA #', 'Site #', 'Site Name'], inplace=True)
#Replacing less than 20 with 19
FreeRed['Reduced'].replace({'less than 20': 19}, inplace=True)
FreeRed['Free'].replace({'less than 20': 19}, inplace=True)
#Getting rid of Total info
FreeRed=FreeRed[~FreeRed['agency_code'].str.contains('Total')]
#setting index
FreeRed.set_index('agency_code', inplace=True)
#dropping State of North Carolina
FreeRed.drop(FreeRed.tail(2).index, inplace=True)

In [35]:
#Race and Gender
RaceGender=pd.read_csv('ec_pupils.csv')
#Rename
RaceGender.columns=RaceGender.columns.str.replace(' ', '_')
#Change total to numeric
RaceGender['Total']=RaceGender['Total'].str.replace(',', '').apply(float)
#Female totals
Female=[]
for i in range(0, len(RaceGender.columns)):
    if (RaceGender.columns[i].endswith("Female")):
        Female.append(RaceGender.columns[i])
RaceGender['Female_Tot']=RaceGender[Female].sum(axis=1).divide(RaceGender['Total'])
#Male totals
Male=[]
for i in range(0, len(RaceGender.columns)):
    if (RaceGender.columns[i].split('_')[-1]=="Male"):
        Male.append(RaceGender.columns[i])
RaceGender['Male_Tot']=RaceGender[Male].sum(axis=1).divide(RaceGender['Total'])
#Indian totals
RaceGender['Indian']=RaceGender[['Indian_Male', 'Indian_Female']].sum(axis=1).divide(RaceGender['Total'])
#Asian totals
RaceGender['Asian']=RaceGender[['Asian_Male', 'Asian_Female']].sum(axis=1).divide(RaceGender['Total'])
#Hispanic totals
RaceGender['Hispanic']=RaceGender[['Hispanic_Male', 'Hispanic_Female']].sum(axis=1).divide(RaceGender['Total'])
#Black totals
RaceGender['Black']=RaceGender[['Black_Male', 'Black_Female']].sum(axis=1).divide(RaceGender['Total'])
#White totals
RaceGender['White']=RaceGender[['White_Male', 'White_Female']].sum(axis=1).divide(RaceGender['Total'])
#Black totals
RaceGender['PacificIsland']=RaceGender[['Pacific_Island_Male', 'Pacific_Island_Female']].sum(axis=1).divide(RaceGender['Total'])
#Multiracial totals
RaceGender['Multiracial']=RaceGender[['Two_or__More_Male', 'Two_or__More_Female']].sum(axis=1).divide(RaceGender['Total'])
#creating agency_code
RaceGender['LEA']=RaceGender['LEA'].apply(str)
RaceGender['LEA'].replace(['10', '20', '30', '40', '50', '60', '70', '80', '90'], 
                         ['010', '020', '030', '040', '050', '060', '070', '080', '090'], inplace=True)
RaceGender['School']=RaceGender['School'].apply(str)
RaceGender['agency_code']=RaceGender['LEA']+RaceGender['School']
#Drop extra columns
RaceGender.drop(columns=['Year', 'LEA', 'LEA_Name', 'School', 'School_Name', 'Indian_Male', 'Indian_Female', 'Asian_Male', 
                         'Asian_Female', 'Hispanic_Male', 'Hispanic_Female', 'Black_Male', 'Black_Female', 'White_Male',
                         'White_Female', 'Pacific_Island_Male', 'Pacific_Island_Female', 'Two_or__More_Male', 
                         'Two_or__More_Female'], inplace=True)
#set index
RaceGender.set_index('agency_code', inplace=True)

In [36]:
#ACT
ACT=pd.read_excel('rcd_acc_act.xlsx')
#keep only year 2017
ACT=ACT[ACT['year']==2017]
#keep findings for all
ACT=ACT[ACT['subgroup']=='ALL']
#Getting rid of LEA and state info
ACT=ACT[~ACT['agency_code'].str.endswith('LEA')]
ACT=ACT[ACT['agency_code']!='00B000']
#Dropping unnecessary columns
ACT.drop(columns=['year', 'subgroup', 'den', 'masking'], inplace=True)
#Making table wide
ACT=ACT.pivot(index='agency_code', columns='subject', values='pct').reset_index()
#dropping State of North Carolina
ACT.drop(ACT.tail(1).index, inplace=True)
#set index
ACT.set_index('agency_code', inplace=True)

In [37]:
#Economically Disadvantaged Youth
EDS=pd.read_excel('rcd_acc_eds.xlsx')
#year 2017
EDS=EDS[EDS['year']==2017]
#drop LEAs
EDS=EDS[~EDS['agency_code'].str.endswith('LEA')]
#drop SEA
EDS=EDS[~EDS['agency_code'].str.endswith('SEA')]
EDS
#Dropping unnecessary columns
EDS.drop(columns=['year', 'pct_eds_masking'], inplace=True)
#set index
EDS.set_index('agency_code', inplace=True)

In [38]:
#School Performance Grades
SPG=pd.read_excel('rcd_acc_spg1.xlsx')
#Selecting year = 2017
SPG=SPG[SPG['year']==2017]
#Dropping unnecessary columns
SPG.drop(columns=['year', 'asm_option', 'subgroup', 'ma_score', 'ma_score_masking', 'ma_grade', 'rd_score', 'rd_score_masking',
                 'rd_grade', 'sc_score', 'sc_score_masking', 'm1_score', 'm1_score_masking', 'e2_score', 'e2_score_masking', 
                  'bi_score', 'bi_score_masking', 'act_score', 'act_score_masking', 'wk_score', 'wk_score_masking', 
                  'mcr_score', 'mcr_score_masking', 'cgrs_score', 'cgrs_score_masking', 'ach_score', 'ach_score_masking'],
        inplace=True)
#set index
SPG.set_index('agency_code', inplace=True)

In [39]:
#Average Daily Membership
ADM=pd.read_excel('rcd_adm.xlsx')
#Selecting year = 2017
ADM=ADM[ADM['year']==2017]
#Dropping columns
ADM.drop(columns=['year','category_code'], inplace=True)
#renaming
ADM.rename(columns={'avg_student_num': 'ADM'}, inplace=True)
#set index
ADM.set_index('agency_code', inplace=True)

In [40]:
#Advanced Placement Info
AP=pd.read_excel('rcd_ap.xlsx')
#selecting year = 2017
AP=AP[AP['year']==2017]
#drop year
AP.drop(columns=['year'], inplace=True)
#change agency code to string
AP.agency_code=AP.agency_code.apply(str).str.zfill(6)
#set index
AP.set_index('agency_code', inplace=True)

In [41]:
#Attendance
ATT=pd.read_excel('rcd_att.xlsx')
#Selecting year = 2017
ATT=ATT[ATT['year']==2017]
#drop category code
ATT.drop(columns=['category_code'], inplace=True)
#set index
ATT.set_index('agency_code', inplace=True)

In [42]:
#Average number of books
AvgBooks=pd.read_excel('AvgNumBookTitles.xlsx')
#Rename columns
AvgBooks.rename(columns={'School Number': 'agency_code'}, inplace=True)
#drop columns
AvgBooks.drop(columns=['School Name', 'LEA Name'], inplace=True)
#change agency code to string
AvgBooks.agency_code=AvgBooks.agency_code.apply(str).str.zfill(6)
#set index
AvgBooks.set_index('agency_code', inplace=True)

#Media Collection Age
MediaAge=pd.read_excel('MediaCollectionAge.xlsx')
#rename columns
MediaAge.rename(columns={'School Number': 'agency_code'}, inplace=True)
#drop columns
MediaAge.drop(columns=['School Name1', 'LEA Name'], inplace=True)
#change agency code to string
MediaAge.agency_code=MediaAge.agency_code.apply(str).str.zfill(6)
#set index
MediaAge.set_index('agency_code', inplace=True)

#School 1-1 program
School1_1=pd.read_excel('School1-1.xlsx')
#rename columns
School1_1.rename(columns={'School Number': 'agency_code'}, inplace=True)
#drop columns
School1_1.drop(columns=['School Name1', 'LEA Name'], inplace=True)
#change agency code to string
School1_1.agency_code=School1_1.agency_code.apply(str).str.zfill(6)
#set index
School1_1.set_index('agency_code', inplace=True)

#WAP
WAP=pd.read_excel('WAP.xlsx')
#rename columns
WAP.rename(columns={'School Number': 'agency_code', '201.  Wireless Access Point Count': 'WAP_Count',
                   '216.  Number of Classrooms':'NumClassrooms', 'APs Per Classroom': 'WAP_per_Classroom'}, inplace=True)
#drop columns
WAP.drop(columns=['School Name1', 'LEA Name'], inplace=True)
#change agency code to string
WAP.agency_code=WAP.agency_code.apply(str).str.zfill(6)
#set index
WAP.set_index('agency_code', inplace=True)

In [43]:
#suspensions and crimes
Incidences=pd.read_excel('rcd_inc1.xlsx')
#keep year=2017
Incidences=Incidences[Incidences['year']==2017]
#drop LEAs
Incidences=Incidences[~Incidences['agency_code'].str.endswith('LEA')]
#drop state
Incidences=Incidences[~Incidences['agency_code'].str.endswith('SEA')]
#drop columns
Incidences.drop(columns=['year', 'category_code'], inplace=True)
#Change to object
Incidences['agency_code']=Incidences['agency_code'].apply(str)
#change agency code to string
Incidences.agency_code=Incidences.agency_code.apply(str).str.zfill(6)
#set index
Incidences.set_index('agency_code', inplace=True)

In [44]:
#per pupil expenditure
Funds=pd.read_excel('rcd_funds.xlsx')
#keep year=2017
Funds=Funds[Funds['year']==2017]
#Drop columns
Funds=Funds[['agency_code', 'total_expense', 'local_perpupil', 'state_perpupil', 'federal_perpupil']]
#keep LEAs (funding is typically assigned locally and you don't have a school by school information piece)
Funds=Funds[Funds['agency_code'].str.endswith('LEA')]
#Creating local rank
Funds["local_rank"] = Funds["local_perpupil"].rank(method ='max', ascending=False) 
#Creating state rank
Funds["state_rank"] = Funds["state_perpupil"].rank(method ='max', ascending=False)
#Creating federal rank
Funds["federal_rank"] = Funds["federal_perpupil"].rank(method ='max', ascending=False) 
#Creating total rank
Funds["total_rank"] = Funds["total_expense"].rank(method ='max', ascending=False) 
#rename agency code since it's really the LEA code
Funds.rename(columns={'agency_code':'LEA'}, inplace=True)
#split LEA
Funds['LEA Number']=Funds['LEA'].astype(str).str[0:3]
#change agency code to string
Funds['LEA Number']=Funds['LEA Number'].apply(str).str.zfill(3)
#drop LEA 
Funds.drop(columns=['LEA'], inplace=True)
#set index
Funds.set_index('LEA Number', inplace=True)

In [45]:
#Class size
Class_Size=pd.read_excel('rcd_sar.xlsx')
#keep year=2017
Class_Size=Class_Size[Class_Size['year']==2017]
#Dropping columns
Class_Size.drop(columns=['year', 'category_code'], inplace=True)
#drop LEAs
Class_Size=Class_Size[~Class_Size['agency_code'].str.endswith('LEA')]
#drop state
Class_Size=Class_Size[~Class_Size['agency_code'].str.endswith('SEA')]
#aggregating average class size
Class_Size=Class_Size.groupby(Class_Size['agency_code']).mean().reset_index()
#set index
Class_Size.set_index('agency_code', inplace=True)

In [46]:
#SAT info
SAT=pd.read_excel('rcd_sat.xlsx')
#keep year=2017
SAT=SAT[SAT['year']=='2017']
#drop LEAs
SAT=SAT[~SAT['agency_code'].str.endswith('LEA')]
#drop state
SAT=SAT[~SAT['agency_code'].str.endswith('SEA')]
#Dropping columns
SAT.drop(columns=['year', 'category_code'], inplace=True)
#change agency code to string
SAT.agency_code=SAT.agency_code.apply(str).str.zfill(6)
#set index
SAT.set_index('agency_code', inplace=True)

In [47]:
#Teacher effectiveness
Teach_Effect=pd.read_excel('rcd_effectiveness.xlsx')
#year 2017
Teach_Effect=Teach_Effect[Teach_Effect['year']==2017]
#Selecting only student growth
Teach_Effect=Teach_Effect[Teach_Effect['ee_standard']=='TS']
#drop columns
Teach_Effect.drop(columns=['year', 'category_code', 'count', 'ee_standard'], inplace=True)
#drop LEAs
Teach_Effect=Teach_Effect[~Teach_Effect['agency_code'].str.endswith('LEA')]
#drop state
Teach_Effect=Teach_Effect[~Teach_Effect['agency_code'].str.endswith('SEA')]
#Making table wide
Teach_Effect=Teach_Effect.pivot(index='agency_code', columns='ee_rating', values='pct_rating').reset_index()
#Removing spaces from column names
Teach_Effect.columns = Teach_Effect.columns.str.replace(' ', '_')
#Replacing missing with 0
Teach_Effect.fillna(0)
#set index
Teach_Effect.set_index('agency_code', inplace=True)

In [48]:
#Teaching experience
Teach_Experience=pd.read_excel('rcd_experience.xlsx')
#year 2017
Teach_Experience=Teach_Experience[Teach_Experience['year']==2017]
#Only teachers
Teach_Experience=Teach_Experience[Teach_Experience['staff']=='Teacher']
#Dropping columns
Teach_Experience.drop(columns=['year', 'category_code', 'staff', 'total_class_teach'], inplace=True)
#drop LEAs
Teach_Experience=Teach_Experience[~Teach_Experience['agency_code'].str.endswith('LEA')]
#drop state
Teach_Experience=Teach_Experience[~Teach_Experience['agency_code'].str.endswith('SEA')]
#set index
Teach_Experience.set_index('agency_code', inplace=True)

In [49]:
#Teaching Licenses
Teach_Licenses=pd.read_excel('rcd_licenses.xlsx')
#year 2017
Teach_Licenses=Teach_Licenses[Teach_Licenses['year']==2017]
#Dropping columns
Teach_Licenses.drop(columns=['year', 'category_code'], inplace=True)
#drop LEAs
Teach_Licenses=Teach_Licenses[~Teach_Licenses['agency_code'].str.endswith('LEA')]
#drop state
Teach_Licenses=Teach_Licenses[~Teach_Licenses['agency_code'].str.endswith('SEA')]
#set index
Teach_Licenses.set_index('agency_code', inplace=True)

In [50]:
#National Board Certified Teachers
Teach_NBPTS=pd.read_excel('rcd_nbpts.xlsx')
#year 2017
Teach_NBPTS=Teach_NBPTS[Teach_NBPTS['year']==2017]
#Dropping columns
Teach_NBPTS.drop(columns=['year', 'category_code', 'total_nbpts_num'], inplace=True)
#drop LEAs
Teach_NBPTS=Teach_NBPTS[~Teach_NBPTS['agency_code'].str.endswith('LEA')]
#drop state
Teach_NBPTS=Teach_NBPTS[~Teach_NBPTS['agency_code'].str.endswith('SEA')]
#set index
Teach_NBPTS.set_index('agency_code', inplace=True)

In [51]:
#Teacher degrees
Teach_Degrees=pd.read_excel('TeacherDegrees.xlsx')
#creating agency_code
Teach_Degrees['agency_code']=Teach_Degrees['LEA#']+Teach_Degrees['Sch#']
#dropping extra columns
Teach_Degrees.drop(columns=['LEA#', 'Sch#', 'LEA Name', 'School Name'], inplace=True)
#change agency code to string
Teach_Degrees.agency_code=Teach_Degrees.agency_code.apply(str).str.zfill(6)
#set index
Teach_Degrees.set_index('agency_code', inplace=True)

In [66]:
#supplement data
supplement=pd.read_excel('Local_Salary.xlsx')
#dropping years
supplement.drop(columns=['Year', 'Teacher No. of Position', 'Teacher No. Rec. Supplmt.', 'Principal No. of Position',
                        'PrincipalNo. Rec. Supplmt.', 'Assistant PrincipalsNo. of Position', 
                        'Assistant PrincipalsNo. Rec. Supplmt.'], inplace=True)
#Renaming columns
supplement.rename(columns={'Teacher Average Supplmt.': 'AvgTeachSuppl',
                           'Principal Average Supplmt.': 'AvgPrincSuppl',
                           'Assistant PrincipalsAverage Supplmt.': 'AvgAssistPrincSuppl',
                           'LEA': 'LEA Number'}, inplace=True)
#Changing LEA number to string
supplement['LEA Number']=supplement['LEA Number'].apply(str).str.zfill(3)
#supplement['LEA Number'].replace(['10', '20', '30', '40', '50', '60', '70', '80', '90'], 
                         #['010', '020', '030', '040', '050', '060', '070', '080', '090'], inplace=True)
#set index
supplement.set_index('LEA Number', inplace=True)

In [67]:
#School info
school=pd.read_csv('custom_report_school.csv')
#rename
school.rename(columns={'School Number': 'agency_code', 'School Calendar Description': 'School Calendar'}, inplace=True)
#selecting only public
school=school[school['School Designation Type']=='P']
#dropping extra columns
school.drop(columns=['School Calendar Type', 'School Designation Type', 'School Designation Desc'], inplace=True)
#change LEA Number  and agency_code to string
school['LEA Number']=school['LEA Number'].apply(str).str.zfill(3)
school['agency_code']=school['agency_code'].apply(str).str.zfill(6)
#set index
school.set_index('LEA Number', inplace=True)

In [68]:
#joining in supplement information
schoolsup=school.join(supplement, on='LEA Number').join(Funds, on='LEA Number')
#Setting index for large joining
schoolsup.set_index('agency_code', inplace=True)

In [69]:
#joining all the tables into one table
tables=[SPG, grad4, grad5, FreeRed, RaceGender, ACT, SAT, EDS, ADM, AP, ATT, AvgBooks, MediaAge, School1_1, WAP, 
        Incidences, Class_Size, Teach_Effect, Teach_Experience, Teach_Licenses, Teach_NBPTS, Teach_Degrees]
total=schoolsup
for table in tables:
    total=total.join(table, on='agency_code')

In [70]:
#Creating percentages
total['FreePct']=total['Free'].divide(total['ADM'])*100
total['ReducedPct']=total['Reduced'].divide(total['ADM'])*100
#Calculating crime per 100 (percentage)
total['crimeper100']=total['crime'].divide(total['ADM'])*100
total['shortsusper100']=total['short_term'].divide(total['ADM'])*100
total['longsusper100']=total['long_term'].divide(total['ADM'])*100
total['expulsionper100']=total['expulsion'].divide(total['ADM'])*100

In [71]:
#Removing spaces from column names
total.columns=total.columns.str.replace(' ', '_')

#splitting locale type
total[['Locale1', 'Locale2']]=total.Locale_Type_Desc.str.split(', ', expand=True) 

#removing comma from locale type
total.Locale_Type_Desc=total.Locale_Type_Desc.str.replace(', ', '')

#removing space from region
total.SBE_Region_Names=total.SBE_Region_Names.str.replace(' ', '')

#Changing values for School1_1
total['School1-1']=total['School1-1'].str.replace('Yes', '1')
total['School1-1']=total['School1-1'].str.replace('No', '0')

#Creating dummy variables for several columns
total=pd.get_dummies(total, prefix=['SBERegion', 'SchoolType', 'Calendar', 'LocaleType', 'Locale', 'Grade'],
               columns=['SBE_Region_Names', 'School_Type', 'School_Calendar', 'Locale_Type_Desc', 'Locale1', 'category_code'], 
                     drop_first=True)

#Removing spaces from column names
total.columns=total.columns.str.replace(' ', '_')

In [72]:
#Writing to a file to use in previous analysis
total.to_csv("NCSchoolRemade.csv")